<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/compare_extractive_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
!pip install rouge-score
!pip install import-ipynb

In [2]:
import pickle
from rouge_score import rouge_scorer
import import_ipynb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
# load in functions from extract_summarization notebook
#%cd "drive/MyDrive/Colab Notebooks"
import extractive_summarization

In [29]:
def paired_bootstrap(evals, models, configs, model1, model2):

   for metric in [('fmeasure', False), ('fmeasure', True), ('precision', False), ('precision', True), ('recall', False), ('recall', True)]:

    # which model is better and by how much
    if evals[model1][metric]['mean'] > evals[model2][metric]['mean']:
      better_model = model1
      other_model = model2
    else:
      better_model = model2
      other_model = model1
    diff =  evals[better_model][metric]['mean'] - evals[other_model][metric]['mean']

    for i in range(10):
      # generate bootstrap samples 
      bs_sample = {}
      bs_sample[model1] = models[model1][metric].sample(n = len(models[model1][metric]), replace = True, random_state = 123)
      bs_sample[model2] = models[model2][metric].sample(n = len(models[model2][metric]), replace = True, random_state = 123)

      # retrain both models on bootstrap samples with the current config....call in functions from other notebook?
        # create a notebook that purely has functions, no calls and separate main into different notebook?
      bs_results = {}
      gt_diff = 0
      lt_diff = 0
      embeddings = extractive_summarization.load_embeddings()
      for m in [model1, model2]:
        config = configs[model1][metric]
        if 'baseline' in config:
          tfidf, feature_array = extractive_summarization.corpus_tfidf(bs_sample[m])
        else:
          tfidf = ''
          feature_array = ''
        eval_results, _ = extractive_summarization.train_config_loop(bs_sample[m], tfidf, feature_array, embeddings, [config])
        bs_results[m] = eval_results[metric]['mean']
   
      # find difference in relevant stat
      diff_bs = bs_results[better_model] - bs_results[other_model]
      if diff_bs >= 2*diff:
        gt_diff += 1
      else:
        lt_diff += 1
      
    # calculate p value
    pvalue = gt_diff / (gt_diff + lt_diff)

    return (metric, best_model, pvalue)

In [54]:
model_dict = {}
eval_dict = {}
config_dict = {}
for model in ['lsa', 'baseline']: # 'baseline']: #'pr']:
  with open('/content/drive/MyDrive/data/trained_model_' + model + '.pkl', 'rb') as f:
    eval_dict[model] = pickle.load(f)
    model_dict[model] = pickle.load(f)
    config_dict[model] = pickle.load(f)

#paired_bootstrap(eval_dict, model_dict, config_dict, 'baseline', 'lsa')

In [ ]:
# TODO stats on types of summaries generated by each model and for best of each metric. Ex: recall best summaries longer than precision summaries? etc.
# problems include that sentences can appear in odd orders. BUT our human content creator can rearrange - care more about getting the right info in front of them.
  # thus prefer longer summaries? care more about recall than precision?

In [ ]:
# TODO compare evaluations between models with paired bootstrap test to test significance?

# for best config of each model type, calculate difference in stat performance (recall etc.)
# generate N bootstrapped samples of data 
# for the best config of each model type, evaluate on replicate data. calculate difference in performance
# count percent of replicate diffs that are >= 2 * original diff = p-value  
  # null hypothesis is that there is no difference and the true effect size is original diff (data happens to be biased towards one model)
  # if see a lot of replicated diffs >= 2 * original diff, then null is true and there is no difference between the models